In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("toyset.csv")
df = df.drop(columns=["ID"])

X_vars = ["VIDEO", "PUZZLE", "FEMALE"]
X = df[X_vars].values
Y = df["ICE_CREAM"].values

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
pd.set_option('display.precision', 6)

In [2]:
from sklearn.linear_model import LogisticRegression
from IPython.display import HTML, display


def coefficient_cmp(multinomial_clf, binary1_clf, binary2_clf):
    mnl, bnl = [], []
        
    m_beta1 = np.append(multinomial_clf.coef_[0], multinomial_clf.intercept_[0])
    m_beta2 = np.append(multinomial_clf.coef_[1], multinomial_clf.intercept_[1])
    m_beta3 = np.append(multinomial_clf.coef_[2], multinomial_clf.intercept_[2])
    b_beta1 = np.append(binary1_clf.coef_[0], binary1_clf.intercept_[0])
    b_beta2 = np.append(binary2_clf.coef_[0], binary2_clf.intercept_[0])
    
    m_beta1 = m_beta1 - m_beta3
    m_beta2 = m_beta2 - m_beta3
    
    columns = ["Beta1", "", "", "", "Beta2", "", "", ""]
    index = ["MNL", "BNL"]
    mnl = np.concatenate((m_beta1, m_beta2))
    bnl = np.concatenate((b_beta1, b_beta2))
    tab = np.stack((mnl, bnl))
    
    df = pd.DataFrame(tab, columns=columns, index=index)
    display(HTML(df.to_html()))
    print("Absolute Coef Diff Sum:", np.sum(np.abs(mnl-bnl)))
    
    
def aggregate_stats(multinomial_clf, binary1_clf, binary2_clf, X, Y):
    P = multinomial_clf.predict_proba(X)
    m = P.mean(axis=0)
    M1 = P[:,0] / (P[:,0] + P[:,2])
    M2 = P[:,1] / (P[:,1] + P[:,2])
    B1 = binary1_clf.predict_proba(X)[:,1]
    B2 = binary2_clf.predict_proba(X)[:,1]
    CP1D = np.abs(M1 - B1)
    CP2D = np.abs(M2 - B2)
    
    tab = []
    col_headers = ["X1","X2","X3","Y","Multinom CP1","Multinom CP2,","Binom CP1","Binom CP2","Abs CP1 Diff","Abs CP2 Diff"]
    row_headers = ["Mean", "Stdev", "Mean(Y=1)", "Stdev(Y=1)", "Mean(Y=2)", "Stdev(Y=2)", "Mean(Y=3)", "Stdev(Y=3)"]
    tab.append([*X.mean(axis=0), Y.mean(axis=0), M1.mean(), M2.mean(), B1.mean(), B2.mean(), CP1D.mean(), CP2D.mean()])
    tab.append([*X.std(axis=0), Y.std(axis=0), M1.std(), M2.std(), B1.std(), B2.std(), CP1D.std(), CP2D.std()])
    for i in range(1, 4):
        idx = np.where(Y == i)
        tab.append([*X[idx].mean(axis=0), Y[idx].mean(axis=0), M1[idx].mean(), M2[idx].mean(), B1[idx].mean(), B2[idx].mean(), CP1D[idx].mean(), CP2D[idx].mean()])
        tab.append([*X[idx].std(axis=0), Y[idx].std(axis=0), M1[idx].std(), M2[idx].std(), B1[idx].std(), B2[idx].std(), CP1D[idx].std(), CP2D[idx].std()])
    
    df = pd.DataFrame(tab, columns=col_headers, index=row_headers)
    display(HTML(df.to_html()))
    
    
def fit_models(X, Y):
    # Multinomial
    multinomial_clf = LogisticRegression(solver="lbfgs", multi_class='multinomial', max_iter=400)
    multinomial_clf.fit(X, Y)

    # Binary
    # 1 vs. 3 (exclude 2)
    idx = np.where(Y != 2)
    X1, Y1 = X[idx], Y[idx]
    Y1[Y1 == 3] = 0
    binary1_clf = LogisticRegression(solver="lbfgs", multi_class='ovr', max_iter=400)
    binary1_clf.fit(X1, Y1)

    # 2 vs. 3 (exclude 1)
    idx = np.where(Y != 1)
    X2, Y2 = X[idx], Y[idx]
    Y2[Y2 == 3] = 0
    binary2_clf = LogisticRegression(solver="lbfgs", multi_class='ovr', max_iter=400)
    binary2_clf.fit(X2, Y2)
        
    # Return observation-level probabilities
    P = multinomial_clf.predict_proba(X)
    return P, multinomial_clf, binary1_clf, binary2_clf

### Compare MNL vs. BNL on original data

In [3]:
P, multinomial_clf, binary1_clf, binary2_clf = fit_models(X, Y)
coefficient_cmp(multinomial_clf, binary1_clf, binary2_clf)
aggregate_stats(multinomial_clf, binary1_clf, binary2_clf, X, Y)

,Beta1,,,,Beta2,,,
MNL,-0.046961,-0.081139,0.769087,6.004344,-0.023112,-0.042894,0.019524,4.068166
BNL,-0.041046,-0.072079,0.714930,5.266007,-0.024158,-0.042844,0.010742,4.124978


Absolute Coef Diff Sum: 0.8741575393727312


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.850000,52.405000,0.545000,2.055000,0.448556,0.641349,0.449335,0.641181,0.018010,0.001719
Stdev,9.876108,10.708920,0.497971,0.722478,0.243522,0.128241,0.224533,0.129590,0.008505,0.001205
Mean(Y=1),47.702128,47.319149,0.680851,1.000000,0.584612,0.706855,0.576092,0.707155,0.020016,0.001714
Stdev(Y=1),10.301652,10.721040,0.466147,0.000000,0.232111,0.115095,0.213277,0.116348,0.008050,0.001053
Mean(Y=2),51.705263,52.031579,0.505263,2.000000,0.451178,0.647643,0.451232,0.647631,0.016297,0.001521
Stdev(Y=2),8.879158,9.920158,0.499972,0.000000,0.223353,0.116660,0.205535,0.117833,0.009006,0.001074
Mean(Y=3),55.448276,57.137931,0.500000,3.000000,0.334010,0.577955,0.343511,0.577156,0.019189,0.002048
Stdev(Y=3),9.709882,9.875864,0.500000,0.000000,0.225814,0.126974,0.208726,0.128462,0.007389,0.001432


In [4]:
def simulate(X, P, choices, N):
    """ Takes an existing set of input data X and associated predicted probabilities (generated by some estimated model).
    Copies X independent variable and re-simulates N examples (spread as evenly as possible among the observations in X). """
    np.random.seed(0)
    X_new, Y_new = [], []
    for i in range(N):
        X_new.append(X[i % len(P)])
        Y_new.append(np.random.choice(choices, p=P[i % len(P)]))
    return np.array(X_new), np.array(Y_new)

### When we take the multinomial model and simulate more data, the coefficients converge. 
### But this is by construction, and the data generated by MNL has the IIA property.

In [5]:
models = []
sizes = [200, 2000, 20000, 200000, 2000000]
for N in sizes:
    print(N)
    X_new, Y_new = simulate(X, P, [1, 2, 3], N)
    _, MNL, BNL1, BNL2 = fit_models(X_new, Y_new)
    aggregate_stats(MNL, BNL1, BNL2, X_new, Y_new)
    models.append([MNL, BNL1, BNL2])

200


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.850000,52.405000,0.545000,2.045000,0.473198,0.663034,0.477118,0.662245,0.017015,0.006279
Stdev,9.876108,10.708920,0.497971,0.709207,0.246996,0.159574,0.257740,0.164423,0.012793,0.005259
Mean(Y=1),48.021739,47.869565,0.673913,1.000000,0.599033,0.728250,0.609632,0.727150,0.018012,0.007288
Stdev(Y=1),10.224627,10.765502,0.468780,0.000000,0.232154,0.145398,0.241024,0.151440,0.014409,0.006431
Mean(Y=2),50.676768,52.666667,0.555556,2.000000,0.495614,0.683463,0.499234,0.684030,0.017260,0.005271
Stdev(Y=2),9.282141,10.636320,0.496904,0.000000,0.241661,0.151020,0.253183,0.154637,0.012614,0.003965
Mean(Y=3),57.163636,55.727273,0.418182,3.000000,0.327603,0.571718,0.326479,0.568747,0.015742,0.007250
Stdev(Y=3),8.318137,9.385588,0.493260,0.000000,0.190532,0.145654,0.198331,0.151543,0.011514,0.005838


2000


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.850000,52.405000,0.545000,2.067500,0.436681,0.619717,0.437148,0.619560,0.008769,0.001487
Stdev,9.876108,10.708920,0.497971,0.734809,0.244894,0.125970,0.249524,0.126256,0.006536,0.000426
Mean(Y=1),47.293501,46.872117,0.643606,1.000000,0.584822,0.688229,0.588132,0.687885,0.008940,0.001370
Stdev(Y=1),9.529267,10.375063,0.478934,0.000000,0.228539,0.112406,0.232167,0.112888,0.006257,0.000435
Mean(Y=2),51.580681,52.441273,0.511526,2.000000,0.435691,0.623101,0.436109,0.623062,0.009026,0.001484
Stdev(Y=2),9.884393,10.202484,0.499867,0.000000,0.236091,0.122319,0.240965,0.122602,0.006652,0.000424
Mean(Y=3),55.802288,56.663399,0.517974,3.000000,0.322691,0.561282,0.321014,0.561094,0.008253,0.001582
Stdev(Y=3),8.404119,9.692044,0.499677,0.000000,0.205453,0.112462,0.209215,0.112742,0.006545,0.000398


20000


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.850000,52.405000,0.545000,2.047800,0.454372,0.641020,0.454289,0.640980,0.001255,0.000892
Stdev,9.876108,10.708920,0.497971,0.725614,0.242191,0.125459,0.242096,0.125701,0.000658,0.000605
Mean(Y=1),47.727859,47.357380,0.665904,1.000000,0.592181,0.706023,0.592153,0.705944,0.001188,0.000818
Stdev(Y=1),9.646763,10.476731,0.471673,0.000000,0.224490,0.111613,0.224319,0.111917,0.000705,0.000588
Mean(Y=2),51.816638,52.091787,0.510293,2.000000,0.453908,0.644677,0.453712,0.644698,0.001315,0.000906
Stdev(Y=2),9.728755,10.396958,0.499894,0.000000,0.232892,0.120197,0.232796,0.120446,0.000659,0.000611
Mean(Y=3),55.343219,57.127645,0.500867,3.000000,0.340172,0.580818,0.340227,0.580709,0.001212,0.000930
Stdev(Y=3),8.922721,9.264338,0.499999,0.000000,0.209480,0.115843,0.209475,0.116068,0.000604,0.000603


200000


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.850000,52.405000,0.545000,2.056755,0.446465,0.639195,0.446482,0.639163,0.000567,0.000236
Stdev,9.876108,10.708920,0.497971,0.723128,0.241763,0.126238,0.242034,0.126166,0.000392,0.000135
Mean(Y=1),47.728791,47.353551,0.672547,1.000000,0.585433,0.704439,0.585641,0.704324,0.000588,0.000228
Stdev(Y=1),9.612180,10.473129,0.469284,0.000000,0.224908,0.111890,0.225144,0.111857,0.000402,0.000135
Mean(Y=2),51.724637,52.035907,0.507602,2.000000,0.446965,0.643847,0.446943,0.643827,0.000582,0.000235
Stdev(Y=2),9.690684,10.421100,0.499942,0.000000,0.232947,0.121294,0.233212,0.121230,0.000398,0.000135
Mean(Y=3),55.372489,57.072861,0.503097,3.000000,0.333745,0.579092,0.333674,0.579108,0.000525,0.000246
Stdev(Y=3),9.025967,9.275381,0.499990,0.000000,0.208158,0.116628,0.208401,0.116556,0.000368,0.000134


2000000


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.850000,52.405000,0.545000,2.055484,0.448006,0.640735,0.448024,0.640732,0.000263,0.000059
Stdev,9.876108,10.708920,0.497971,0.722653,0.242783,0.127492,0.243021,0.127489,0.000161,0.000012
Mean(Y=1),47.711099,47.322216,0.669390,1.000000,0.587680,0.706831,0.587841,0.706814,0.000290,0.000053
Stdev(Y=1),9.593776,10.474379,0.470433,0.000000,0.226051,0.112953,0.226268,0.112958,0.000173,0.000013
Mean(Y=2),51.712801,52.038186,0.508184,2.000000,0.448938,0.645533,0.448948,0.645534,0.000263,0.000059
Stdev(Y=2),9.703660,10.416111,0.499933,0.000000,0.233831,0.122426,0.234065,0.122425,0.000162,0.000012
Mean(Y=3),55.422378,57.116261,0.504558,3.000000,0.333495,0.579424,0.333410,0.579427,0.000242,0.000063
Stdev(Y=3),8.991853,9.246450,0.499979,0.000000,0.208250,0.117562,0.208451,0.117558,0.000145,0.000010


In [6]:
for i in range(len(sizes)):
    print(sizes[i])
    coefficient_cmp(*models[i])

200


,Beta1,,,,Beta2,,,
MNL,-0.081258,-0.046518,0.931824,6.000387,-0.073853,-0.002964,0.370338,4.554908
BNL,-0.081965,-0.057365,0.909113,6.638242,-0.078704,0.000024,0.349441,4.664013


Absolute Coef Diff Sum: 0.8099598750793463
2000


,Beta1,,,,Beta2,,,
MNL,-0.062259,-0.073347,0.534873,6.454311,-0.037467,-0.028017,-0.087531,3.987327
BNL,-0.067441,-0.071826,0.600525,6.605140,-0.037671,-0.027976,-0.101710,4.003111


Absolute Coef Diff Sum: 0.25339009728659756
20000


...

Absolute Coef Diff Sum: 0.06330733068317326
200000


,Beta1,,,,Beta2,,,
MNL,-0.045674,-0.080530,0.790407,5.883550,-0.021927,-0.042504,0.027958,3.969607
BNL,-0.045508,-0.080772,0.797095,5.883832,-0.021989,-0.042425,0.025661,3.969728


Absolute Coef Diff Sum: 0.00993748930612684
2000000


,Beta1,,,,Beta2,,,
MNL,-0.046555,-0.080936,0.766076,5.971881,-0.022743,-0.042712,0.021470,4.034820
BNL,-0.046536,-0.081103,0.768526,5.978286,-0.022746,-0.042709,0.020909,4.035141


Absolute Coef Diff Sum: 0.009928262263711553


### How can we modify the data so that the coeefficients CANNOT converge to each other?

### What if there is overlap between Y=1 and Y=2? Test this by copy N% of examples from Y=2 and relabeling them as Y=1. Copying a higher % (ranging from 20-100) does NOT seem to affect the difference between conditional probability.

In [7]:
def copy_2to1(X, Y, ratio):
    idx2 = np.where(Y == 2)[0]
    X_new = np.concatenate((X, X[idx2[:int(ratio*len(idx2))]]))
    Y_new = np.concatenate((Y, np.ones(int(ratio*len(idx2)))))
    return X_new, Y_new

n = 200000
print(n)
models = []
X_new, Y_new = simulate(X, P, [1, 2, 3], n)
ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
    print(ratio)
    X_new_c, Y_new_c = copy_2to1(X_new, Y_new, ratio)
    _, MNL, BNL1, BNL2 = fit_models(X_new_c, Y_new_c)
    aggregate_stats(MNL, BNL1, BNL2, X_new_c, Y_new_c)
    models.append([MNL, BNL1, BNL2])

200000
0.2


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.841734,52.378710,0.542251,1.965276,0.532987,0.639518,0.532794,0.639503,0.001796,0.001188
Stdev,9.861293,10.683974,0.498212,0.752296,0.210660,0.125625,0.208912,0.125751,0.000978,0.000277
Mean(Y=1),48.886800,48.719253,0.626723,1.000000,0.618078,0.686501,0.617140,0.686712,0.001857,0.001099
Stdev(Y=1),9.809197,10.674474,0.483675,0.000000,0.198369,0.117900,0.196831,0.117933,0.000943,0.000266
Mean(Y=2),51.724637,52.035907,0.507602,2.000000,0.535808,0.643915,0.535653,0.643827,0.001761,0.001192
Stdev(Y=2),9.690684,10.421100,0.499942,0.000000,0.203166,0.121141,0.201462,0.121230,0.000974,0.000274
Mean(Y=3),55.372489,57.072861,0.503097,3.000000,0.432209,0.579260,0.432799,0.579108,0.001786,0.001283
Stdev(Y=3),9.025967,9.275381,0.499990,0.000000,0.191700,0.116403,0.190005,0.116556,0.001019,0.000260


0.4


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.827300,52.351441,0.539038,1.888369,0.594855,0.639673,0.595135,0.639866,0.002174,0.001807
Stdev,9.851041,10.665985,0.498474,0.767581,0.187327,0.123826,0.185089,0.125415,0.001367,0.001224
Mean(Y=1),49.506465,49.460323,0.598843,1.000000,0.653369,0.676371,0.652915,0.677182,0.002071,0.001821
Stdev(Y=1),9.859547,10.714375,0.490133,0.000000,0.177363,0.118643,0.175347,0.120034,0.001199,0.001120
Mean(Y=2),51.724637,52.035907,0.507602,2.000000,0.598405,0.643657,0.598704,0.643827,0.002123,0.001745
Stdev(Y=2),9.690684,10.421100,0.499942,0.000000,0.180897,0.119730,0.178725,0.121230,0.001353,0.001198
Mean(Y=3),55.372489,57.072861,0.503097,3.000000,0.503909,0.579779,0.505226,0.579108,0.002406,0.001889
Stdev(Y=3),9.025967,9.275381,0.499990,0.000000,0.175939,0.114937,0.173693,0.116556,0.001578,0.001394


0.6


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.820422,52.324768,0.536574,1.822816,0.642197,0.639976,0.642504,0.640186,0.001935,0.002132
Stdev,9.839827,10.647270,0.498661,0.774366,0.168570,0.123313,0.166569,0.125114,0.001342,0.001423
Mean(Y=1),49.913228,49.922236,0.581825,1.000000,0.685343,0.670367,0.685112,0.671159,0.001821,0.002132
Stdev(Y=1),9.867768,10.703030,0.493259,0.000000,0.160206,0.119299,0.158399,0.120923,0.001170,0.001319
Mean(Y=2),51.724637,52.035907,0.507602,2.000000,0.645802,0.643647,0.646117,0.643827,0.001887,0.002067
Stdev(Y=2),9.690684,10.421100,0.499942,0.000000,0.163008,0.119527,0.161073,0.121230,0.001319,0.001394
Mean(Y=3),55.372489,57.072861,0.503097,3.000000,0.559503,0.579886,0.560753,0.579108,0.002217,0.002239
Stdev(Y=3),9.025967,9.275381,0.499990,0.000000,0.161667,0.114708,0.159573,0.116556,0.001600,0.001628


0.8


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.811445,52.301608,0.534436,1.766270,0.679556,0.640239,0.679827,0.640480,0.001663,0.002208
Stdev,9.827540,10.630573,0.498813,0.775734,0.153349,0.123004,0.151655,0.124845,0.001228,0.001472
Mean(Y=1),50.187551,50.241178,0.570033,1.000000,0.712954,0.666291,0.712837,0.667037,0.001562,0.002205
Stdev(Y=1),9.856699,10.682393,0.495071,0.000000,0.146141,0.119639,0.144608,0.121330,0.001079,0.001382
Mean(Y=2),51.724637,52.035907,0.507602,2.000000,0.682961,0.643617,0.683237,0.643827,0.001623,0.002144
Stdev(Y=2),9.690684,10.421100,0.499942,0.000000,0.148523,0.119483,0.146887,0.121230,0.001204,0.001445
Mean(Y=3),55.372489,57.072861,0.503097,3.000000,0.603685,0.579882,0.604763,0.579108,0.001942,0.002318
Stdev(Y=3),9.025967,9.275381,0.499990,0.000000,0.149352,0.114660,0.147518,0.116556,0.001494,0.001677


1


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,51.809694,52.286332,0.532976,1.716996,0.709821,0.640422,0.710062,0.640662,0.001382,0.002245
Stdev,9.817048,10.618633,0.498911,0.773561,0.140473,0.122749,0.139028,0.124620,0.001087,0.001499
Mean(Y=1),50.401119,50.485001,0.562236,1.000000,0.736449,0.663180,0.736404,0.663865,0.001297,0.002238
Stdev(Y=1),9.846030,10.668458,0.496112,0.000000,0.134253,0.119849,0.132937,0.121589,0.000962,0.001417
Mean(Y=2),51.724637,52.035907,0.507602,2.000000,0.713004,0.643616,0.713246,0.643827,0.001346,0.002184
Stdev(Y=2),9.690684,10.421100,0.499942,0.000000,0.136246,0.119449,0.134849,0.121230,0.001064,0.001473
Mean(Y=3),55.372489,57.072861,0.503097,3.000000,0.639907,0.579900,0.640845,0.579108,0.001646,0.002362
Stdev(Y=3),9.025967,9.275381,0.499990,0.000000,0.138424,0.114624,0.136829,0.116556,0.001340,0.001710


In [8]:
for i in range(len(ratios)):
    print(ratios[i])
    coefficient_cmp(*models[i])

0.2


,Beta1,,,,Beta2,,,
MNL,-0.037555,-0.068211,0.538418,5.395019,-0.022037,-0.042352,0.014296,3.974601
BNL,-0.037001,-0.067647,0.524533,5.342707,-0.021989,-0.042425,0.025661,3.969728


Absolute Coef Diff Sum: 0.08367389600904418
0.4


,Beta1,,,,Beta2,,,
MNL,-0.034006,-0.061921,0.401757,5.244613,-0.021606,-0.041883,0.012300,3.926529
BNL,-0.033365,-0.061214,0.386102,5.182266,-0.021989,-0.042425,0.025661,3.969728


Absolute Coef Diff Sum: 0.1368367375456979
0.6


...

Absolute Coef Diff Sum: 0.13650874435925536
0.8


,Beta1,,,,Beta2,,,
MNL,-0.029972,-0.055702,0.270209,5.178541,-0.021537,-0.041795,0.008609,3.919921
BNL,-0.029491,-0.055178,0.256769,5.132423,-0.021989,-0.042425,0.025661,3.969728


Absolute Coef Diff Sum: 0.12850454505670433
1


,Beta1,,,,Beta2,,,
MNL,-0.028718,-0.053848,0.236439,5.184704,-0.021530,-0.041783,0.008289,3.918996
BNL,-0.028351,-0.053348,0.224677,5.144806,-0.021989,-0.042425,0.025661,3.969728


Absolute Coef Diff Sum: 0.12173291077953233


### What if we add noise to the simulated data? Take a fixed % of class 1 (here we use 30%), and "rotate" that many examples from 1->2->3->1. Then there is a noticeable increase in the coefficient difference.

In [9]:
def rotate(Y, ratio):
    idx1 = np.where(Y == 1)[0]
    idx2 = np.where(Y == 2)[0]
    idx3 = np.where(Y == 3)[0]
    M = int(ratio*len(idx1))
    Y_new = np.copy(Y)
    Y_new[idx1[:M]] = 2
    Y_new[idx2[:M]] = 3
    Y_new[idx3[:M]] = 1
    return Y_new

print(n)
X_new, Y_new = simulate(X, P, [1, 2, 3], n)
_, MNL, BNL1, BNL2 = fit_models(X_new, Y_new)

print("Before rotation:")
coefficient_cmp(MNL, BNL1, BNL2)

Y_new = rotate(Y_new, 0.3)
_, MNL, BNL1, BNL2 = fit_models(X_new, Y_new)

print("After rotation:")
coefficient_cmp(MNL, BNL1, BNL2)

200000
Before rotation:


,Beta1,,,,Beta2,,,
MNL,-0.045674,-0.080530,0.790407,5.883550,-0.021927,-0.042504,0.027958,3.969607
BNL,-0.045508,-0.080772,0.797095,5.883832,-0.021989,-0.042425,0.025661,3.969728


Absolute Coef Diff Sum: 0.00993748930612684
After rotation:


,Beta1,,,,Beta2,,,
MNL,-0.024499,-0.043004,0.460549,3.088268,-0.018521,-0.034865,0.091902,3.287670
BNL,-0.023356,-0.041462,0.453318,2.950914,-0.019213,-0.035904,0.104453,3.373407


Absolute Coef Diff Sum: 0.24728831848011834


### What if we want to go from non-IIA to IIA, other than by simulating more examples? Here we test out duplicating occurrences of class 3. However, it seems that limiting P(Y=1) and P(Y=2) does not make the coefficients any closer, when compared to normal simulations of the same population size.

In [10]:
def dup3(X, Y, N):
    X_new, Y_new = np.copy(X), np.copy(Y)
    idx3 = np.where(Y == 3)[0]
    while len(X_new) < N:
        X_new = np.append(X_new, X[idx3], axis=0)
        Y_new = np.append(Y_new, Y[idx3], axis=0)
    return X_new[:N], Y_new[:N]

models = []
sizes = [200, 2000, 20000, 200000, 2000000]
for N in sizes:
    print(N)
    X_new, Y_new = simulate(X, P, [1, 2, 3], int(N/2))
    X_new, Y_new = dup3(X_new, Y_new, N)
    _, MNL, BNL1, BNL2 = fit_models(X_new, Y_new)
    aggregate_stats(MNL, BNL1, BNL2, X_new, Y_new)
    models.append([MNL, BNL1, BNL2])

200


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,52.210000,52.785000,0.545000,2.480000,0.184441,0.311607,0.189320,0.311685,0.008071,0.004464
Stdev,9.684828,10.006936,0.497971,0.713863,0.170037,0.113945,0.179331,0.114943,0.009348,0.003621
Mean(Y=1),45.961538,45.846154,0.730769,1.000000,0.340774,0.399110,0.352597,0.397576,0.012585,0.006281
Stdev(Y=1),10.017219,10.636267,0.443560,0.000000,0.206580,0.121900,0.215873,0.124237,0.012039,0.004211
Mean(Y=2),50.019231,52.230769,0.576923,2.000000,0.220790,0.339959,0.228155,0.340513,0.009845,0.004139
Stdev(Y=2),9.973986,11.046432,0.494047,0.000000,0.184181,0.123123,0.194321,0.123809,0.010138,0.003520
Mean(Y=3),54.475410,54.500000,0.491803,3.000000,0.135631,0.280875,0.137971,0.281093,0.006353,0.004215
Stdev(Y=3),8.630116,8.643438,0.499933,0.000000,0.125260,0.093473,0.133168,0.094729,0.007740,0.003409


2000


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,53.967500,54.334000,0.521000,2.526500,0.166461,0.274321,0.167415,0.274803,0.002527,0.002848
Stdev,9.383147,10.495592,0.499559,0.702352,0.166353,0.103864,0.169981,0.107063,0.002945,0.002517
Mean(Y=1),46.819672,46.729508,0.643443,1.000000,0.332851,0.368466,0.337411,0.371335,0.005247,0.004342
Stdev(Y=1),9.905303,10.299241,0.478982,0.000000,0.215574,0.119485,0.220347,0.123820,0.004262,0.003668
Mean(Y=2),51.886710,52.947712,0.520697,2.000000,0.196320,0.297423,0.197887,0.298716,0.002883,0.003284
Stdev(Y=2),9.527407,10.144785,0.499571,0.000000,0.176799,0.107309,0.180753,0.110822,0.003275,0.002790
Mean(Y=3),56.048574,56.255204,0.498072,3.000000,0.124591,0.248435,0.124651,0.248180,0.001889,0.002413
Stdev(Y=3),8.349162,9.911502,0.499996,0.000000,0.124864,0.085718,0.127531,0.088207,0.002078,0.001950


20000


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,53.712050,54.864600,0.524400,2.524850,0.167595,0.283843,0.167701,0.283856,0.000804,0.000614
Stdev,9.519876,10.290659,0.499404,0.697483,0.160930,0.122824,0.161508,0.123125,0.000920,0.000514
Mean(Y=1),47.672712,47.284690,0.658372,1.000000,0.311211,0.386241,0.311807,0.386385,0.001315,0.000742
Stdev(Y=1),9.587089,10.411257,0.474256,0.000000,0.200106,0.133186,0.200917,0.133581,0.001207,0.000606
Mean(Y=2),51.700903,51.984667,0.513758,2.000000,0.206181,0.318894,0.206474,0.319015,0.000999,0.000695
Stdev(Y=2),9.787214,10.392728,0.499811,0.000000,0.172708,0.126348,0.173394,0.126699,0.001063,0.000573
Mean(Y=3),55.568931,57.326780,0.503652,3.000000,0.126857,0.252008,0.126804,0.251956,0.000637,0.000560
Stdev(Y=3),8.764516,9.216243,0.499987,0.000000,0.125807,0.104268,0.126194,0.104513,0.000734,0.000461


200000


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,53.616920,54.749600,0.523025,2.526825,0.165213,0.280798,0.165179,0.280770,0.000072,0.000212
Stdev,9.615318,10.298957,0.499470,0.696958,0.155604,0.114172,0.155553,0.114071,0.000098,0.000184
Mean(Y=1),47.757686,47.347274,0.674589,1.000000,0.301973,0.372844,0.301894,0.372690,0.000121,0.000273
Stdev(Y=1),9.595413,10.424861,0.468528,0.000000,0.192497,0.122459,0.192435,0.122355,0.000130,0.000233
Mean(Y=2),51.729537,52.057118,0.506939,2.000000,0.198367,0.311167,0.198322,0.311120,0.000088,0.000231
Stdev(Y=2),9.711413,10.421255,0.499952,0.000000,0.164662,0.117047,0.164607,0.116950,0.000107,0.000198
Mean(Y=3),55.383453,57.094402,0.501147,3.000000,0.127981,0.252783,0.127958,0.252785,0.000058,0.000194
Stdev(Y=3),9.006808,9.298316,0.499999,0.000000,0.124806,0.098771,0.124764,0.098688,0.000082,0.000164


2000000


,X1,X2,X3,Y,Multinom CP1,"Multinom CP2,",Binom CP1,Binom CP2,Abs CP1 Diff,Abs CP2 Diff
Mean,53.643092,54.762086,0.524446,2.528059,0.164481,0.281456,0.164546,0.281456,0.000219,0.000054
Stdev,9.613535,10.275815,0.499402,0.695568,0.156097,0.116058,0.156338,0.116082,0.000277,0.000048
Mean(Y=1),47.702070,47.303910,0.669385,1.000000,0.302929,0.376067,0.303201,0.376094,0.000387,0.000067
Stdev(Y=1),9.587903,10.468027,0.470435,0.000000,0.194370,0.125025,0.194690,0.125052,0.000382,0.000059
Mean(Y=2),51.702666,52.040263,0.508788,2.000000,0.198938,0.312941,0.199059,0.312946,0.000265,0.000059
Stdev(Y=2),9.700650,10.418572,0.499923,0.000000,0.165969,0.119262,0.166243,0.119286,0.000315,0.000052
Mean(Y=3),55.436554,57.118676,0.503859,3.000000,0.126644,0.252680,0.126651,0.252673,0.000172,0.000049
Stdev(Y=3),8.993004,9.240824,0.499985,0.000000,0.124124,0.099899,0.124302,0.099917,0.000219,0.000043


In [11]:
for i in range(len(sizes)):
    print(sizes[i])
    coefficient_cmp(*models[i])

200


,Beta1,,,,Beta2,,,
MNL,-0.061949,-0.064216,1.214711,4.068920,-0.050857,-0.002043,0.406101,1.698847
BNL,-0.067000,-0.067341,1.190485,4.514441,-0.053315,0.000129,0.363909,1.735131


Absolute Coef Diff Sum: 0.5610265965222861
2000


,Beta1,,,,Beta2,,,
MNL,-0.080065,-0.067621,0.589115,5.597782,-0.044282,-0.018042,0.001447,2.338665
BNL,-0.082236,-0.068995,0.603024,5.770626,-0.046221,-0.018036,-0.017642,2.452096


Absolute Coef Diff Sum: 0.3247615623922261
20000


...

Absolute Coef Diff Sum: 0.07972586431570203
200000


,Beta1,,,,Beta2,,,
MNL,-0.045282,-0.080844,0.806491,4.391326,-0.021983,-0.042398,0.032175,2.477195
BNL,-0.045348,-0.080773,0.805923,4.391175,-0.022079,-0.042281,0.030197,2.476929


Absolute Coef Diff Sum: 0.003312712178394076
2000000


,Beta1,,,,Beta2,,,
MNL,-0.046695,-0.081339,0.769213,4.504749,-0.023043,-0.042707,0.025437,2.557040
BNL,-0.046598,-0.081596,0.770160,4.512554,-0.023014,-0.042740,0.025852,2.557077


Absolute Coef Diff Sum: 0.009618840805404361
